# Creating a rotating globe to show instagram coverage

Following this [blog](https://makersportal.com/blog/2018/8/16/rotating-globe-in-python-using-basemap-toolkit)

In [ ]:
output_dir = '../../images/instagram-coverage-globe/'
output_file = 'globe-coverage-22places.mp4'

png_dir = output_dir + 'png_dir/'
data_file = '../../data/wikivoyage/enriched/wikivoyage_destinations.csv'

In [ ]:
import os
os.environ['PROJ_LIB'] = '<path to anaconda>/anaconda3/share/proj'

In [ ]:
import datetime,matplotlib,time
from mpl_toolkits.basemap import Basemap
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import shutil
import imageio

### Step 0: get the places that we featured

In [ ]:
# hard copy from excel instagram post list
featured_places = [522962, 212363, 733660, 311565, 334786, 749803, 749803, 698560, 734084, 179384, 762363, 440744, 892966, 539442, 767322, 595111, 644651, 258755, 464720, 145126, 627255, 981575]

In [ ]:
df = pd.read_csv(data_file).loc[lambda df: df['id'].isin(featured_places)][['lat', 'lng']]
points = [(v['lat'], v['lng']) for (k, v) in df.iterrows()]

### Step 1: Create function for plotting 3D globe

In [ ]:
# set perspective angle
lat_viewing_angle = 10
lon_viewing_angle = 9
coordinates = [(lat_viewing_angle, lon_viewing_angle)]

def create_image(lat, lon, points, filename, show_plot=False):
    fig = plt.figure(figsize=(9,6))

    # define color maps for water and land
    ocean_map = (plt.get_cmap('ocean'))(210)
    cmap = plt.get_cmap('gist_earth')

    # call the basemap and use orthographic projection at viewing angle
    m = Basemap(projection='ortho', 
              lat_0=lat, lon_0=lon)    

    # coastlines, map boundary, fill continents/water, fill ocean, draw countries
    m.bluemarble(scale=0.2)   # full scale will be overkill
    m.drawcoastlines(color='white', linewidth=0.2)  # add coastlines
#     m.drawmapboundary(fill_color=ocean_map)
#     m.fillcontinents(color=cmap(200),lake_color=ocean_map)
#     m.drawcountries()

    # latitude/longitude line vectors
    # lat_line_range = [-90,90]
    # lat_lines = 8
    # lat_line_count = (lat_line_range[1]-lat_line_range[0])/lat_lines

    # merid_range = [-180,180]
    # merid_lines = 8
    # merid_count = (merid_range[1]-merid_range[0])/merid_lines

    # m.drawparallels(np.arange(lat_line_range[0],lat_line_range[1],lat_line_count))
    # m.drawmeridians(np.arange(merid_range[0],merid_range[1],merid_count))
    
    # scatter to indicate lat/lon point
    for i in points:
        lat = i[0]
        lon = i[1]
        x,y = m(lon,lat)
    #     m.scatter(x,y,marker='o',color='#DDDDDD',s=3000,zorder=10,alpha=0.7, edgecolor='#000000')
        m.scatter(x, y, marker='o', color='#F16C21', s=50, alpha = 1)

        # For neon effect, redraw the data with low alpha and slighty increased linewidth:
        n_shades = 10
        diff_linewidth = 1.05
        alpha_value = 0.5 / n_shades
        for n in range(1, n_shades+1):
            m.scatter(x, y, marker='o', color='#F16C21', s=100,
                      linewidth=2+(diff_linewidth*n), alpha=alpha_value)

    # save figure at 150 dpi and show it
    plt.savefig(filename, dpi=150, transparent=True)
    
    if show_plot:
        plt.show()
        
    plt.close()

In [ ]:
create_image(lat_viewing_angle, lon_viewing_angle, 
             points, 
             output_dir + 'orthographic_map_example_python.png', True)

## Step 2: Rotate earth and save pngs

In [ ]:
rotation_steps = 100

# create vector of longitutes to loop over
lon_viewing_angle = [-180, 180]
lon_vec = np.linspace(lon_viewing_angle[0],lon_viewing_angle[1],rotation_steps)
start_index = min(range(len(lon_vec)), key=lambda i: abs(lon_vec[i]+lat_viewing_angle))
lon_steps = np.hstack([lon_vec[start_index:],lon_vec[:start_index]])

In [ ]:
# remove old images before placing new
if os.path.exists(png_dir):
    shutil.rmtree(png_dir)
os.makedirs(png_dir)

# loop through the longitude vector above
gif_indx = 0
for lon in lon_steps:    
    create_image(31, lon, points, png_dir + 'frame_'+str(gif_indx)+'_.png')
    gif_indx+=1

## Step 3: Make Gif/Movie

In [ ]:
def gif_maker(gif_name, png_dir):
    # define some GIF parameters
    frame_length = 0.5 # seconds between frames
    end_pause = 4 # seconds to stay on last frame
    # define MP4 parameters
    fps = 20

    # list the images for in the animation
    images,image_file_names = [],[]
    for file_name in os.listdir(png_dir):
        if file_name.endswith('.png'):
            image_file_names.append(file_name)       
    sorted_files = sorted(image_file_names, key=lambda y: int(y.split('_')[1]))
    
    # loop through files, join them to image array, and write to GIF called 'wind_turbine_dist.gif'
    for ii in range(0,len(sorted_files)):       
        file_path = os.path.join(png_dir, sorted_files[ii])
        if ii==len(sorted_files)-1:
            for jj in range(0,int(end_pause/frame_length)):
                images.append(imageio.imread(file_path))
        else:
            images.append(imageio.imread(file_path))

    if gif_name.split('.')[-1] == 'gif':
        # the duration is the time spent on each image (1/duration is frame rate)
        imageio.mimsave(gif_name, images,'GIF',duration=frame_length)
    else:  # make mp4
        writer = imageio.get_writer(gif_name, fps=fps)
        for im in [png_dir + file for file in sorted_files]:
            writer.append_data(imageio.imread(im))
        writer.close()


In [ ]:
# gif_maker(output_dir + 'test.gif', output_dir + 'png_dir/')
gif_maker(output_dir + output_file, png_dir)

Done.